### Speech dataset

In [43]:
# audio root
audio_root = "/Users/lukas/Desktop/Projects/MIT/data/peoples_speech/audio_debug"

In [52]:
from src.data.components.datasets import AudioDataset

dataset = AudioDataset(root_dir=audio_root)

Dataset found 5 flac files
Counting samples...
Counting took 0.6520729064941406 seconds
Dataset has 386 samples


In [53]:
item = dataset[1]
item.keys()

dict_keys(['input_values'])

In [54]:
item["input_values"].shape

(153307,)

## Multimodal Dataset

In [11]:
csv_path = "/Users/lukas/Desktop/Projects/MIT/data/peoples_speech/debug/files.csv"

import pandas as pd

df = pd.read_csv(csv_path)

audio_files = df.iloc[:, 1].tolist()  # First column
text_files = df.iloc[:, 2].tolist()  # Second column

In [12]:
from src.data.components.datasets import PeoplesMultiModalDataset

dataset = PeoplesMultiModalDataset(
    alignment_files=text_files,
    flac_files=audio_files,
)

Loading files took 0.12194991111755371 seconds
Dataset has 977777 samples


In [13]:
audio, text = dataset.__getitem__(1)

curr_txt_offset: 0, len: 2610


In [14]:
# create dataloader

from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

for i, batch in enumerate(dataloader):
    print(i, batch)
    if i > 100:
        break

curr_txt_offset: 90, len: 2610
0 [tensor([[-0.0302, -0.1455,  0.1164,  ..., -0.2841,  0.1874, -0.1072]],
       dtype=torch.float64), ('',)]
curr_txt_offset: 180, len: 2610
1 [tensor([[ 0.0446, -0.2279,  0.2324,  ...,  0.2914,  0.2877,  0.2301]],
       dtype=torch.float64), ('',)]
curr_txt_offset: 270, len: 2610
2 [tensor([[-0.1417, -0.1721, -0.2033,  ..., -0.2817, -0.1458, -0.1779]],
       dtype=torch.float64), ('',)]
curr_txt_offset: 360, len: 2610
3 [tensor([[ 0.2317, -0.2409, -0.2714,  ...,  0.3201, -0.1947,  0.0715]],
       dtype=torch.float64), ('',)]
curr_txt_offset: 450, len: 2610
4 [tensor([[ 0.0645, -0.0316, -0.1630,  ...,  0.0160,  0.0107,  0.0298]],
       dtype=torch.float64), ('',)]
curr_txt_offset: 540, len: 2610
5 [tensor([[0.0962, 0.0623, 0.0779,  ..., 0.1184, 0.2311, 0.2968]],
       dtype=torch.float64), ('',)]
curr_txt_offset: 630, len: 2610
6 [tensor([[ 0.0730,  0.0884,  0.1078,  ..., -0.0422, -0.0482, -0.0522]],
       dtype=torch.float64), ('',)]
curr_txt_offs

In [16]:
def _get_text_from_dict(cnt: list, start_idx: int, end_idx: int):
    print(f"cnt: {cnt}")
    return " ".join([c["word"] for c in cnt[start_idx:end_idx]])


cnt = [
    {
        "start": 351.947,
        "end": 352.007,
        "word": "to",
        "confidence": 0.918,
        "break_before": 0.01999999999998181,
        "duration": 0.060000000000002274,
    },
    {
        "start": 352.087,
        "end": 352.287,
        "word": "put",
        "confidence": 0.918,
        "break_before": 0.07999999999998408,
        "duration": 0.19999999999998863,
    },
    {
        "start": 352.327,
        "end": 352.847,
        "word": "distillate",
        "confidence": 0.657,
        "break_before": 0.040000000000020464,
        "duration": 0.5199999999999818,
    },
    {
        "start": 352.867,
        "end": 353.188,
        "word": "property.",
        "confidence": 0.931,
        "break_before": 0.020000000000038654,
        "duration": 0.32099999999996953,
    },
]

print(_get_text_from_dict(cnt, 0, 3))

cnt: [{'start': 351.947, 'end': 352.007, 'word': 'to', 'confidence': 0.918, 'break_before': 0.01999999999998181, 'duration': 0.060000000000002274}, {'start': 352.087, 'end': 352.287, 'word': 'put', 'confidence': 0.918, 'break_before': 0.07999999999998408, 'duration': 0.19999999999998863}, {'start': 352.327, 'end': 352.847, 'word': 'distillate', 'confidence': 0.657, 'break_before': 0.040000000000020464, 'duration': 0.5199999999999818}, {'start': 352.867, 'end': 353.188, 'word': 'property.', 'confidence': 0.931, 'break_before': 0.020000000000038654, 'duration': 0.32099999999996953}]
to put distillate


## Run with packs

In [19]:
import pandas as pd
from src.data.components.datasets import (
    PeoplesMultiModalDataset,
    PeoplesMultiModalPackagedDataset,
)

In [20]:
mapping_path = "/Users/lukas/Desktop/Projects/MIT/data/peoples_speech/debug/files.csv"
mapping = pd.read_csv(mapping_path)
alignment_files = mapping.iloc[:, 2].tolist()
flac_files = mapping.iloc[:, 1].tolist()

# Initialize the parameters
sr = 16000  # sample rate
num_words_per_sample = 50  # number of words per sample
dataset_total_words = 100000  # total words in the dataset
samples_per_pack = 100  # number of samples per pack
save_dir = "/Users/lukas/Desktop/Projects/MIT/data/peoples_speech/packs/debug"  # directory to save the sample pack

In [23]:
# Instantiate the dataset
dataset = PeoplesMultiModalDataset(
    alignment_files=alignment_files,
    flac_files=flac_files,
    sr=sr,
    num_words_per_sample=num_words_per_sample,
    dataset_total_words=dataset_total_words,
)

# Wrap the dataset with the packaged dataset class
packaged_dataset = PeoplesMultiModalPackagedDataset(
    dataset=dataset,
    num_files_per_pack=samples_per_pack,
    save_dir=save_dir,
)

Loading 10 alignment files
Dataset has 2000 samples


In [24]:
# dataloader batch size 1

from torch.utils.data import DataLoader

dataloader = DataLoader(packaged_dataset, batch_size=1, shuffle=True)

for i, batch in enumerate(dataloader):
    print(batch)

[tensor([[-0.0363, -0.0314, -0.0170,  ...,  0.0032,  0.0021,  0.0007]],
       dtype=torch.float64), ("and was really born out of precedent that was preexisting at the time, the holding is explaining why quorum novus is not available in the military courts. But there's also an alternative holding that directly addresses the issue that Petitioner raised here, which is whether or not is to apply",)]
[tensor([[-0.0213, -0.0287, -0.0243,  ..., -0.0350,  0.0348, -0.0430]],
       dtype=torch.float64), ("metallic sound was heard consistent with the gun. After he ran away, within minutes thereafter, a firearm was recovered from the same place. His fingerprints weren't on the gun, is that right? That's right, Your Honor. It had one, Your Honor, but the testimony that's in the record shows that",)]
[tensor([[-0.0117, -0.0152, -0.0117,  ..., -0.0650, -0.0623, -0.0606]],
       dtype=torch.float64), ("I looked at the scope of services that you put together for Framingham, and your name was includ

Exception: Pack file does not exist. Please prepare the packs first.